# Exploration du jeu de données 

*Pour récupérer le Dataset, on a automatisé ça en récupérant le token de l'API Kaggle*  

*Pour récupérer le Dataset :*  
- Aller sur [Kaggle](https://www.kaggle.com/).  
- Se connecter à son compte ou s'inscrire.  
- Cliquer en haut à droite sur l'icône de son profil.  
- Aller dans la rubrique API de l'onglet settings puis "create new token".    

*Le token de l'API Kaggle est maintenant téléchargé.*  

*Le jeu de données est déjà très propre nous n'avons pas à le clean*